In [1]:
# ライブラリをインポート
import shutil

from google.cloud import speech # 文字起こしに利用するライブラリ
import io
import os
import glob
import ffmpeg
import pandas as pd

# 音声ファイルをカットする際に利用するライブラリ
from numpy import int16
import wave
import struct
import math
import numpy as np

from pydub import AudioSegment
import numpy as np
from scipy.io.wavfile import write

import soundfile # audio file のbit数を変換するために利用するライブラリ

# keyを指定
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../Transcription/key/credentials.json'

In [11]:
# 対象ファイル←ここで指定するだけで効率的にファイルの指定ができるようにする
wave_file_name = '20220411322717_140250_59883_20220411322717.wav'

In [12]:
# 複数のオーディオがあるフォルダ内のwavファイルのpathを取得する
def audio_file_path_getter(check_path):
    path_name = glob.glob(check_path)
    print('pathの数は===', len(path_name))
    return path_name

In [13]:
# audioのファイル名を取得する関数
def audio_name_getter(audio_folder_path):
    name_list = []
    file_name = os.listdir(audio_folder_path)
    for item in file_name:
        # pathから名前のところだけを取得する
        name_list.append(os.path.basename(item).split('.', 1)[0])
    return name_list

In [14]:
# いただいたファイルの音声ファイルのbit数が8bitだったので変換時に16bitに変換しろとエラーが出た
# bit数を変換する関数
def bit_change(audio_file_path, subtype):
    data, fs = soundfile.read(audio_file_path)
    soundfile.write('../Transcription/16bit_audio/bit_change_audio.wav', data, fs, subtype=subtype)
    print('bit数の変換ファイルの出力が終了しました。')


In [15]:
# wavファイルを59秒間隔で分割する関数
def cut_wav(audio_chang_wav, time, wav_cut_dir):
    wr = wave.open(audio_chang_wav, "r")

    # wav情報を取得
    ch = wr.getnchannels()
    width = wr.getsampwidth()
    fr = wr.getframerate()
    fn = wr.getnframes()
    total_time = 1.0 * fn / fr
    integer = math.floor(total_time)
    t = int(time)
    frames = int(ch * fr * t)
    # 小数点切り上げ（1分に満たない最後のシーンを出力するため）
    num_cut = int(math.ceil(integer / t))
    data = wr.readframes(wr.getnframes())
    wr.close()

    X = np.frombuffer(data, dtype=int16)

    for i in range(num_cut):
        outf = wav_cut_dir + str(i) + ".wav"
        start_cut = int(i * frames)
        end_cut = int(i * frames + frames)
        print(start_cut)
        print(end_cut)
        Y = X[start_cut:end_cut]
        outd = struct.pack("h" * len(Y), *Y)

        # 書き出し
        ww = wave.open(outf, "w")
        ww.setnchannels(ch)
        ww.setsampwidth(width)
        ww.setframerate(fr)
        ww.writeframes(outd)
        ww.close()
    print('音声のカットを終了します')

In [16]:
# timestampを本来の時間に戻す関数
def calc_audio_time(dataframe):
    for index, row in dataframe.iterrows():
        # audioを59秒ごとにカットしているので、audio_numの番号 * 59秒で本来の時間に戻してtimestampを更新
        dataframe.at[index, 'new_time_stamp'] = row['new_time_stamp'] + (int(row['audio_num']) * 59)
    return  dataframe

In [17]:
# ステレオチャンネルの音声の文字起こしをする関数
def transcription_wav_stereo(audio_file, box, string_data):
    client = speech.SpeechClient()

    with io.open(audio_file, 'rb') as f:
        content = f.read()

    audio = speech.RecognitionAudio(content=content)

    config = speech.RecognitionConfig(
        encoding = speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz = 8000,
        language_code = 'ja-JP',
        audio_channel_count = 2,
        enable_separate_recognition_per_channel = True,
        # Trueにすると拡張モデルを利用する。何も指定しなかったら通常のモード（有料です）
        use_enhanced=True,
        # 機械学習モデルを選択できるので、それで電話通話を選択する。
        model="phone_call",
    )

    response = client.recognize(config=config, audio=audio)

    for i, result in enumerate(response.results):
        alternative = result.alternatives[0]
        print('-'*20)
        #print('現在文字起こし中です')
        # print('first alternative of result {}'.format(i))
        # print(u'Transcript:{}'.format(alternative.transcript))
        # print(u'Channel Tag:{}'.format(result.channel_tag))
        # print('second is ===', result.result_end_time)
        item = [alternative.transcript, result.channel_tag, result.result_end_time, string_data]
        box.append(item)

    print('文字起こしを終了します。')
    return box

In [18]:
# 音声取得〜整形〜文字起こしまでを一括できるように関数化してみた
def main():
    # 音声ファイルが全部でいくつあるのかを確認して、それぞれのpathを全て取得する
    raw_audio_folder = '../Transcription/audio/*.wav'
    audio_file = audio_file_path_getter(raw_audio_folder)
    print(audio_file)

    # 音声ファイルの名前だけを取得する（最後にテキストファイルの名前に利用する）
    raw_audio_name = '../Transcription/audio/'
    audio_name = audio_name_getter(raw_audio_name)
    audio_name = list(filter(None, audio_name))
    print(audio_name)
    # # テスト用
    audio_file = ['../Transcription/audio/{}'.format(wave_file_name)]
    audio_name = ['sample_create_1']
    # pathの数だけ実行する
    for audio, name in zip(audio_file, audio_name):
        # 各音声ファイルの出力結果を格納するリスト 音声ファイルのpathが更新される度、空に戻すためにここに配置
        text_save_list = []
        print(audio)
        # audioのbit数を16bitに変換する
        bit_change(audio, 'PCM_16')
        # 音声ファイルを59秒ごとに切り分けてfileに出力する
        audio_file = '../Transcription/16bit_audio/bit_change_audio.wav'
        save_cut_wav = '../Transcription/test_file/'
        cut_wav(audio_file, 59, save_cut_wav)

        # cutしたaudioファイルのpathを取得する
        cut_audio = '../Transcription/test_file/*.wav'
        cut_audio_file = audio_file_path_getter(cut_audio)

        # 文字起こしを実行する
        # 本来はlen(cut_audio_file)
        for item in range(0,2):

            #これで指定しないと、wavファイルの順番がバラバラで読み込まれてしまう
            now_wav_file ='../Transcription/test_file/'+str(item) +'.wav'
            # ステレオチャンネルの文字起こしを実行
            transcription_wav_stereo(now_wav_file, text_save_list, item)

        # 文字起こしした結果をdataframeに変換する
        result_df = pd.DataFrame(text_save_list, columns=['text', 'tag', 'timestamp', 'audio_num'])

        # 出力されたタイムスタンプを整数値に変換する
        result_df['new_time_stamp'] = result_df['timestamp'].dt.total_seconds()
        # timestampの値を本来の時間に戻す
        result_df = calc_audio_time(result_df)

        result_df.to_csv('../Transcription/text_csv/transcription_'+str(name)+'.csv')

        # カットしたaudioを削除する
        for item in cut_audio_file:
            os.remove(item)

        os.remove(audio_file) # 16bitに変換したオーディオファイルも削除しておく

        print('音声ファイルの文字起こしを終了しました！')

    print('実行を終了します')


In [19]:
main()

pathの数は=== 1
['../Transcription/audio/20220411322717_140250_59883_20220411322717.wav']
['20220411322717_140250_59883_20220411322717', '']
../Transcription/audio/20220411322717_140250_59883_20220411322717.wav
bit数の変換ファイルの出力が終了しました。
0
944000
944000
1888000
1888000
2832000
2832000
3776000
3776000
4720000
4720000
5664000
5664000
6608000
6608000
7552000
7552000
8496000
8496000
9440000
音声のカットを終了します
pathの数は=== 10
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
文字起こしを終了します。
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
----------

In [ ]:
#~~~~~ここより下は今のところ使う予定がないコード~~~~~~~~~~~~~~

In [ ]:
#いきなり長いファイルは処理が長くなってしまうので、適当に30秒ほどのデータを作成するときに利用
def create_test_audio_file(start_second, end_second):
    sound = AudioSegment.from_file('../Transcription/audio/{}'.format(wave_file_name), format='wav')
    cut_sound = sound[start_second:end_second]
    cut_sound.export('test_audio.wav', format='wav')

In [ ]:
# ステレオチャンネルの音声をleft, rightで分割して出力する関数
def stereo_channel_split(stereo_audio):
    sound = AudioSegment.from_file(stereo_audio)
    # チャンネル数
    channel_count = sound.channels
    # frame_rate = speech to text でいうところのhertz
    frames_per_second = sound.frame_rate
    # ファイルの音声の長さ（秒）
    duration = sound.duration_seconds
    # 音声ファイルをnumpyで変換
    sound_array = np.array(sound.get_array_of_samples())
    # 右左に分割
    left_sound = sound_array[0:len(sound_array):2]
    right_sound = sound_array[1:len(sound_array):2]
    # 音声データの書き出し
    write('left_audio.wav', frames_per_second, left_sound)
    write('right_audio.wav', frames_per_second, right_sound)
    print('音声の分割が終了しました')

In [ ]:
# 文字起こし関数(モノラルチャンネルver)
def transcription_wav(audio_file, box, string_data):
    # 音声ファイルの読み込み
    with io.open(audio_file, 'rb') as f:
        content = f.read()

    # APIパラメータの作成
    audio = speech.RecognitionAudio(content = content)
    config = speech.RecognitionConfig(
        # 都度エンコーディングする場合は、LINEAR16 しない場合は、ENCODING_UNSPECIFIED
        # hertzはwavファイルによっては変更する必要があるかもしれないです。自分の場合は32000に指定しろと言われました
        encoding = speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz = 8000,
        language_code = "ja-JP",
        #enable_word_time_offsets=True
    )

    # APIの呼び出し
    client = speech.SpeechClient()
    response = client.recognize(config = config, audio = audio)

    # 結果の表示
    # for result in response.results:
    #     print(result.alternatives[0].transcript)
    for i, result in enumerate(response.results):
        # print(result)
        # print(i)
        alternative = result.alternatives[0]
        print('-'*20)
        print('現在文字起こし中です')
        # print('first alternative of result {}'.format(i))
        # print(u'Transcript:{}'.format(alternative.transcript))
        # print(u'Channel Tag:{}'.format(result.channel_tag))
        # print('second is ===', result.result_end_time)
        item = [format(alternative.transcript), result.result_end_time, string_data]
        box.append(item)
    print('文字起こしを終了します')
    return box